In [ ]:
import numpy as np

In [ ]:
def py_cpu_softnms(boxes, scores, max_num_boxes, iou_threshold=0.5, sigma=0.5, thresh=0.001, method=2):
    """
    py_cpu_softnms
    :param boxes:   boexs format [y1, x1, y2, x2]
    :param scores:     boxes scores
    :param max_num_boxes: maximum number of boxes
    :param iou_threshold:     iou threshold
    :param sigma:  gaussian distribution sigma
    :param thresh: final threshold
    :param method: method using (switcher between normal NMS and soft NMS)
    :return:       index of boxes
    """

    # indexes concatenate boxes with the last column
    N = boxes.shape[0]
    indexes = np.array([np.arange(N)])
    boxes = np.concatenate((boxes, indexes.T), axis=1)

    # the order of boxes coordinate is [y1,x1,y2,x2]
    y1 = boxes[:, 0]
    x1 = boxes[:, 1]
    y2 = boxes[:, 2]
    x2 = boxes[:, 3]
    scores = scores
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    for i in range(N):
        # intermediate parameters for later parameters exchange
        tBD = boxes[i, :].copy()
        tscore = scores[i].copy()
        tarea = areas[i].copy()
        pos = i + 1

        #
        if i != N-1:
            maxscore = np.max(scores[pos:], axis=0)
            maxpos = np.argmax(scores[pos:], axis=0)
        else:
            maxscore = scores[-1]
            maxpos = 0
        if tscore < maxscore:
            boxes[i, :] = boxes[maxpos + i + 1, :]
            boxes[maxpos + i + 1, :] = tBD
            tBD = boxes[i, :]

            scores[i] = scores[maxpos + i + 1]
            scores[maxpos + i + 1] = tscore
            tscore = scores[i]

            areas[i] = areas[maxpos + i + 1]
            areas[maxpos + i + 1] = tarea
            tarea = areas[i]

        # IoU calculate
        xx1 = np.maximum(boxes[i, 1], boxes[pos:, 1])
        yy1 = np.maximum(boxes[i, 0], boxes[pos:, 0])
        xx2 = np.minimum(boxes[i, 3], boxes[pos:, 3])
        yy2 = np.minimum(boxes[i, 2], boxes[pos:, 2])

        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        ovr = inter / (areas[i] + areas[pos:] - inter)

        # Three methods: 1.linear 2.gaussian 3.original NMS
        if method == 1:  # linear
            weight = np.ones(ovr.shape)
            weight[ovr > iou_threshold] = weight[ovr > iou_threshold] - ovr[ovr > iou_threshold]
        elif method == 2:  # gaussian
            weight = np.exp(-(ovr * ovr) / sigma)
        else:  # original NMS
            weight = np.ones(ovr.shape)
            weight[ovr > iou_threshold] = 0

        scores[pos:] = weight * scores[pos:]

    # select the boxes and keep the corresponding indexes
    inds = boxes[:, 4][scores > thresh]
    keep = inds.astype(int)

    return keep